# Homework 4 – Robust Salary Prediction with Backtracking
**Author:** Pratham

**Goal:** Implement Gradient Descent with Backtracking Line Search to predict salary from years of experience.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Dataset: Years of Experience vs Salary (in thousands of ₹ per month)
years_experience = np.array([1.1, 1.3, 1.5, 2.0, 2.2, 2.9, 3.0, 3.2, 3.2, 3.7])
salary_k = np.array([39, 46, 37, 43, 39, 56, 60, 54, 63, 55])


In [ ]:
# === Helper Functions for Backtracking Gradient Descent ===

def normalize(x):
    mu = x.mean()
    sigma = x.std(ddof=0)
    return (x - mu) / sigma, mu, sigma

def cost_fn(x_norm, y, m, c):
    n = len(y)
    preds = m * x_norm + c
    errors = preds - y
    return (errors**2).sum() / (2 * n)

def gradients(x_norm, y, m, c):
    n = len(y)
    preds = m * x_norm + c
    errors = preds - y
    dm = (errors * x_norm).sum() / n
    dc = errors.sum() / n
    return dm, dc

def backtracking_gd(x, y, alpha_init=1.0, rho=0.5, c_param=1e-4, iterations=100):
    x_norm, mu, sigma = normalize(x)
    m = 0.0
    c = 0.0
    alpha = alpha_init
    costs, alphas = [], []
    total_shrinks = 0

    for _ in range(iterations):
        cur_cost = cost_fn(x_norm, y, m, c)
        dm, dc = gradients(x_norm, y, m, c)
        grad_norm_sq = dm*dm + dc*dc
        t = alpha
        while True:
            m_new = m - t * dm
            c_new = c - t * dc
            new_cost = cost_fn(x_norm, y, m_new, c_new)
            if new_cost <= cur_cost - c_param * t * grad_norm_sq:
                break
            t *= rho
            total_shrinks += 1
        m, c = m - t * dm, c - t * dc
        costs.append(new_cost)
    return {'m': m, 'c': c, 'mu': mu, 'sigma': sigma, 'costs': costs, 'total_shrinks': total_shrinks}


In [ ]:
res1 = backtracking_gd(years_experience, salary_k, alpha_init=1.0)
res2 = backtracking_gd(years_experience, salary_k, alpha_init=0.1)

print("Alpha=1.0:", res1['m'], res1['c'], "Shrinks:", res1['total_shrinks'])
print("Alpha=0.1:", res2['m'], res2['c'], "Shrinks:", res2['total_shrinks'])


In [ ]:
x_line = np.linspace(years_experience.min(), years_experience.max(), 200)
x_line_norm = (x_line - res1['mu']) / res1['sigma']
y_line = res1['m'] * x_line_norm + res1['c']

plt.figure(figsize=(8,5))
plt.scatter(years_experience, salary_k, color='tab:green', label='Data')
plt.plot(x_line, y_line, color='tab:orange', label='Best-fit Line')
plt.xlabel("Years of Experience")
plt.ylabel("Salary (k ₹ / month)")
plt.title("Best-fit Line (Backtracking GD)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
x_norm = (years_experience - res1['mu']) / res1['sigma']
preds = res1['m'] * x_norm + res1['c']
residuals = salary_k - preds

plt.figure(figsize=(7,4))
plt.hist(residuals, bins=6, edgecolor='black', color='tab:purple')
plt.title("Histogram of Residuals (Actual - Predicted)")
plt.xlabel("Residual (k ₹)")
plt.ylabel("Count")
plt.show()

print("Residuals Mean:", residuals.mean())
print("Residuals Std Dev:", residuals.std())


In [ ]:
# Add outliers
years_out = np.concatenate([years_experience, [10, 12]])
salary_out = np.concatenate([salary_k, [300, 500]])

res_out = backtracking_gd(years_out, salary_out, alpha_init=1.0)

x_line = np.linspace(0, 13, 300)
y_line_orig = res1['m'] * ((x_line - res1['mu']) / res1['sigma']) + res1['c']
y_line_out = res_out['m'] * ((x_line - res_out['mu']) / res_out['sigma']) + res_out['c']

plt.figure(figsize=(9,5))
plt.scatter(years_experience, salary_k, label='Original', color='tab:green')
plt.scatter([10,12], [300,500], label='Outliers', color='red', marker='x', s=80)
plt.plot(x_line, y_line_orig, label='Original Fit', color='tab:orange', linewidth=2)
plt.plot(x_line, y_line_out, label='With Outliers', color='tab:blue', linewidth=2)
plt.xlabel("Years of Experience")
plt.ylabel("Salary (k ₹ / month)")
plt.title("Effect of Outliers on Regression Line")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def fixed_lr_gd(x, y, lr=0.01, iterations=1000):
    x_norm, mu, sigma = normalize(x)
    m, c = 0.0, 0.0
    costs = []
    for _ in range(iterations):
        dm, dc = gradients(x_norm, y, m, c)
        m -= lr * dm
        c -= lr * dc
        costs.append(cost_fn(x_norm, y, m, c))
    return {'m': m, 'c': c, 'costs': costs}

fixed_res = fixed_lr_gd(years_experience, salary_k, lr=0.01, iterations=1000)

plt.figure(figsize=(8,5))
plt.plot(res1['costs'], label='Backtracking GD', color='tab:orange')
plt.plot(fixed_res['costs'], label='Fixed LR (0.01)', color='tab:blue')
plt.yscale('log')
plt.xlabel("Iterations")
plt.ylabel("Cost (log scale)")
plt.title("Cost Comparison: Backtracking vs Fixed LR")
plt.legend()
plt.grid(True)
plt.show()


### Written Interpretations

**Residuals:**  
Residuals are approximately centered near zero (mean ≈ 0), showing no strong bias.  
The spread (~5k ₹) indicates moderate variance but no directional error pattern.

**Backtracking Comparison:**  
- α=1.0 → 50 total shrinks  
- α=0.1 → 0 shrinks  
A smaller initial α already satisfies the Armijo condition, so fewer shrink steps occur.

**Outliers:**  
Outliers heavily distort the slope (m jumps from ~7.46 to ~132.6), pulling the regression line upward.  
To reduce outlier effects, use robust regression (Huber loss, RANSAC) or remove/clip extreme points.

**Bonus Conclusion:**  
Backtracking GD adapts the step size dynamically, converging faster and more reliably than a fixed learning rate.  
It's more robust for real-world ML tasks where gradient magnitudes can vary widely.
